In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
os.listdir("../input/dfnet6/pytorch/default/1")

['best.pth']

In [2]:
import gc
import torch

torch.cuda.empty_cache()
torch.cuda.ipc_collect()
gc.collect()

import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

class EnsembleNet(nn.Module):
    def __init__(self, num_classes=10):
        super(EnsembleNet, self).__init__()
        self.num_classes = num_classes

        vgg = models.vgg19(pretrained=False)
        self.vgg = vgg.features

        inception = models.inception_v3(pretrained=False, aux_logits=True)
        inception.fc = nn.Identity()
        self.inception = inception

        resnet = models.resnet50(pretrained=False)
        self.resnet = nn.Sequential(*list(resnet.children())[:-2])

        densenet = models.densenet121(pretrained=False)
        self.densenet = densenet.features

        self.vgg_head = nn.Linear(512, num_classes)
        self.google_head = nn.Linear(2048, num_classes)
        self.resnet_head = nn.Linear(2048, num_classes)
        self.dense_head = nn.Linear(1024, num_classes)

        self.fusion_net = nn.Sequential(
            nn.Linear(512 + 2048 + 2048 + 1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, num_classes)
        )
        
    def flatten(self, out):
        return F.adaptive_avg_pool2d(out, (1, 1)).squeeze(-1).squeeze(-1)

    def forward(self, x):
        vgg_feat = self.flatten(self.vgg(x))  # [B, 512, H/32, W/32]
        inception_feat = self.inception(x)
        if(self.training):
            inception_feat =  inception_feat.logits
        resnet_feat = self.flatten(self.resnet(x))  # [B, 2048, H/32, W/32]
        dense_feat = self.flatten(self.densenet(x))  # [B, 1024, H/32, W/32]
        
        # print(vgg_feat.shape, inception_feat, resnet_feat.shape, dense_feat.shape)
        fused_feat = torch.cat([vgg_feat, inception_feat, resnet_feat, dense_feat], dim=1)

        vgg_out = self.vgg_head(vgg_feat)
        google_out = self.google_head(inception_feat)
        resnet_out = self.resnet_head(resnet_feat)
        dense_out = self.dense_head(dense_feat)
        fusion_out = self.fusion_net(fused_feat)

        return [fusion_out, vgg_out, google_out, resnet_out, dense_out]


In [3]:
import os
from PIL import Image

import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class MyDataset(Dataset):
    def __init__(self, path = "../input/cassava-leaf-disease-classification", transform = None):
        data = pd.read_csv(path + "/train.csv")
        self.images = [path+"/train_images/"+p for p in data["image_id"]]
        self.labels = [x for x in data["label"]]
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = self.transform(Image.open(img_path).convert("RGB"))
        return image, self.labels[idx]

In [4]:
import os
cuda = torch.cuda.is_available()
device = torch.device('cuda' if cuda else 'cpu')

In [5]:
path = "/kaggle/input/cassava-leaf-disease-classification/"
image_path = path+"test_images/"
used_models_keras = {"DEFNet":"../input/dfnet10/pytorch/default/1/model12.pth"}
submission_df = pd.DataFrame(columns=["image_id","label"])
submission_df["image_id"] = os.listdir(image_path)
submission_df["label"] = 0

In [6]:
onlykeras = False
cuda = torch.cuda.is_available()
device = torch.device('cuda' if cuda else 'cpu')
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((512, 512)),          
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

def cut_crop_image(processed_img):
    image = transform(processed_img)
    return image.unsqueeze(0)

def read_preprocess_file(img_path, normalize=False):
    image = Image.open(img_path)
    return (image.size[0], image.size[1]), image

def multi_predict_tfhublayer(img_path, modelinstance):
    img = cut_crop_image(read_preprocess_file(img_path, True)[1]).cuda()
    total = modelinstance(img)
    yhat = total[0].detach().cpu().numpy()
    for i in range(1,len(total)):
        yhat += total[i].detach().cpu().numpy()
    return np.mean(yhat, axis=0)

def predict_and_vote(image_list, modelinstances, onlykeras):
    predictions = [] 
    with tqdm(total=len(image_list)) as process_bar:       
      for img_path in image_list:
        process_bar.update(1)  
        Yhats = np.vstack([func(img_path, modelinstance) for func, modelinstance in modelinstances])
        if onlykeras:
            predictions.append(np.argmax(np.sum(Yhats, axis=0)))
        else:
            predictions.append(Yhats)    
    return predictions

inference_models = []

if "DEFNet" in used_models_keras:
    model = EnsembleNet(num_classes=5).to(device)
    model_pth = used_models_keras["DEFNet"]
    model_pth = torch.load(model_pth, map_location=device)
    model.load_state_dict(model_pth)
    model.eval()
    inference_models.append([multi_predict_tfhublayer, model])

submission_df["label"] = predict_and_vote([image_path+id for id in submission_df["image_id"].values], inference_models, onlykeras)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00,  1.05it/s]


In [7]:
for i in range(len(submission_df["label"])):
    submission_df.loc[i,"label"] = np.argmax(submission_df["label"][i][0], axis=0)
submission_df[["image_id","label"]].to_csv("submission.csv", index=False)
!head submission.csv

image_id,label
2216849948.jpg,4


In [8]:
model = EnsembleNet(num_classes=5).to(device)
model_pth = used_models_keras["DEFNet"]
model_pth = torch.load(model_pth, map_location=device)
model.load_state_dict(model_pth)
model.eval()
0

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


0

In [9]:
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns

classes_name = ['CBB', 'CBSD', 'CGM', 'CMD', 'Healthy']

def getConfusionMatrix(n_labels, n_predicts, class_names, title):
    # 建立資料夾
    folder_path = './matrix'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"資料夾 '{folder_path}' 已建立")

    # 計算 confusion matrix
    cm = confusion_matrix(n_labels, n_predicts)

    # 畫圖
    plt.rcParams["font.size"] = 16  # 全局字體大小（軸標籤等）
    fig, ax = plt.subplots(figsize=(10, 8))  # 可依類別數調整大小

    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues, ax=ax, values_format='d')  # 'd' 表示整數格式
    disp.im_.colorbar.remove()  # 若不想顯示 colorbar 可以移除

    # 自訂軸標籤與標題大小
    ax.set_title(title, fontsize=20)
    ax.set_xlabel("Predicted label", fontsize=16)
    ax.set_ylabel("True label", fontsize=16)
    ax.tick_params(labelsize=14)

    # 放大每個 cell 的文字
    for text in disp.text_.flat:
        text.set_fontsize(18)

    plt.tight_layout()
    plt.savefig(f"{folder_path}/confusion_matrix.png")
    plt.close()

In [10]:
path = "../input/cassava-leaf-disease-classification"
data = pd.read_csv(path + "/train.csv")
fold = pd.read_csv("../input/cut-patch" + "/validation_data.csv")
data = pd.merge(data, fold, on='image_id')
indexes = data["fold"]==0
data = data[indexes].reset_index(drop=True)
images = [path+"/train_images/"+p for p in data["image_id"]]
labels = [x for x in data["label"]]

In [11]:
from PIL import Image
from torchvision import transforms

n_predicts = []
n_labels = labels

transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

with torch.no_grad():
    for i in range(len(labels)):#
        img_path = images[i]
        image = torch.unsqueeze(transform(Image.open(img_path).convert("RGB")), 0)
        image= image.to(device)
        outputs = model(image)
        output = outputs[0]
        for j in range(1,len(outputs)):
            output = output + outputs[j]
        _, predicted = torch.max(output, 1)
        n_predicts.append(predicted.item())

In [12]:
getConfusionMatrix(n_labels, n_predicts, classes_name, "DEFNet")

資料夾 './matrix' 已建立
